In [2]:
import numpy as np
import pandas as pd 
import os 
from random import *

In [3]:
print(os.getcwd())
os.chdir('/Users/tommy84729/python/Dataset')

/Users/tommy84729/python


### 匯入資料（train & target dataset）並將date資料移除 ###

In [4]:
train=pd.read_csv("dataset_X.csv").iloc[ : , 1: ]
target=pd.read_csv("dataset_T.csv").iloc[ : , 1: ]

### 分別看一下train和target的資料狀況，以及變數的狀況 ###

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 17 columns):
AMB_TEMP      1096 non-null float64
CH4           1096 non-null float64
CO            1096 non-null float64
NMHC          1096 non-null float64
NO            1096 non-null float64
NO2           1096 non-null float64
NOx           1096 non-null float64
O3            1096 non-null float64
PM10          1096 non-null float64
RAINFALL      1096 non-null float64
RH            1096 non-null float64
SO2           1096 non-null float64
THC           1096 non-null float64
WD_HR         1096 non-null float64
WIND_DIREC    1096 non-null float64
WIND_SPEED    1096 non-null float64
WS_HR         1096 non-null float64
dtypes: float64(17)
memory usage: 145.7 KB


In [6]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 1 columns):
PM2.5    1096 non-null float64
dtypes: float64(1)
memory usage: 8.7 KB


#### 可以看到train和target兩筆資料集中都沒有遺漏值的部分， 另外在train中總共有17項解釋變數 ####

## 2.1.(a) ##
## Please evaluate the corresponding RMS error on the training set and validation set. 
## In the feature selection stage, please apply polynomials of order M = 1 and M = 2 over the dimension D = 17 of input data.

$$Poly\ (M=2) \ : \ y(x,w) = \omega_0 + \sum_{i=1}^{D}
 \omega_i^2x_i^2 + \sum_{i=1}^{D}\sum_{j=1}^{D}x_{ij}x_ix_j \ $$
 
$$ Error\ : \  E(w) = \frac{1}{2N}\sum_{n=1}^{N}{\{y(x_n,w) - t_n\}} $$ 

### 定義phi函數來計算phi(x)，包含了0次、1次和2次的部分 ###

$$ \Phi_nj = \phi(x_n) = \begin{bmatrix} \phi_0(x_1) & \phi_1(x_1) & ... & \phi_{M-1}(x_1) \\ \phi_0(x_2) & \phi_1(x_2) & ... & \phi_{M-1}(x_2) \\  . & .& ...&. \\ . & .& ...&. \\ \phi_0(x_N) & \phi_1(x_N) & ... & \phi_{M-1}(x_N) \\
\end{bmatrix}$$

$$ W_{ML} = (\Phi^{\intercal}\Phi)^{-1}\Phi^{\intercal}Y $$ 

## 定義切割函數

In [11]:
def split(n) : 
    a=list(range(0,1096))
    index = list()
    shuffle(a)
    num = 1096 // n
    for i in range(n-1) : 
        b = a[num*i:num*(i+1)]
        index.append(b)
    c = a[num*(n-1):]
    index.append(c)
    return index

In [12]:
def phi(x, m ,n) : 
    index = split(n)
    phi = list()
    ans = pd.DataFrame()
    for i in range(n) : 
        cv = x.iloc[index[i]]
        train = x.loc[~x.index.isin(index[i])]
        if m <= 1 :  
            od0_t = pd.DataFrame(np.repeat(1, len(train)))
            od0_cv = pd.DataFrame(np.repeat(1, len(cv)))
            od1_t = pd.DataFrame(train)
            od1_cv = pd.DataFrame(cv)
            od0_t.reset_index(drop = True, inplace = True)
            od0_cv.reset_index(drop = True, inplace = True)
            od1_t.reset_index(drop = True, inplace = True)
            od1_cv.reset_index(drop = True, inplace = True)
            phi_t = pd.concat([od0_t,od1_t], axis = 1 , ignore_index= True)
            phi_cv = pd.concat([od0_cv,od1_cv], axis = 1 , ignore_index= True)
            phi.append(phi_t)
            phi.append(phi_cv)
            
        elif m >1 :
            od0_t = pd.DataFrame(np.repeat(1, len(train)))
            od0_cv = pd.DataFrame(np.repeat(1, len(cv)))
            od1_t = pd.DataFrame(train)
            od1_cv = pd.DataFrame(cv)
            od2_t = pd.DataFrame()
            od2_cv = pd.DataFrame()
            for k in range(len(train)) : 
                d = []
                trains = pd.DataFrame([train.iloc[k,:]])
                for i in range(trains.shape[1]) : 
                    for j in range(trains.shape[1]) : 
                        if j >= i :        
                            num = trains.iloc[0,i]*trains.iloc[0,j]
                            d.append(num)
                d = pd.DataFrame(d).transpose()
                od2_t = pd.concat([od2_t, d], axis = 0, ignore_index=True)
            for k in range(len(cv)) : 
                d = []
                cvs = pd.DataFrame([cv.iloc[k,:]])
                for i in range(cvs.shape[1]) : 
                    for j in range(cvs.shape[1]) : 
                        if j >= i :        
                            num = cvs.iloc[0,i]*cvs.iloc[0,j]
                            d.append(num)
                d = pd.DataFrame(d).transpose()
                od2_cv = pd.concat([od2_cv, d], axis = 0, ignore_index=True)
            od0_t.reset_index(drop = True, inplace = True)
            od0_cv.reset_index(drop = True, inplace = True)
            od1_t.reset_index(drop = True, inplace = True)
            od1_cv.reset_index(drop = True, inplace = True)
            od2_t.reset_index(drop = True, inplace = True)
            od2_cv.reset_index(drop = True, inplace = True)           
            phi_t = pd.concat([od0_t,od1_t,od2_t], axis = 1 , ignore_index= True)
            phi_cv = pd.concat([od0_cv,od1_cv,od2_cv], axis = 1 , ignore_index= True)
            phi.append(phi_t)
            phi.append(phi_cv)
    return  [index , phi]

### 定義估計函數，包含了係數的估計以及RMSE的計算 ###

In [28]:
def estimate(phi, target) : 
    ans = pd.DataFrame()
    index = phi[0]
    phi = phi[1]
    for i in range(len(phi)//2) : 
        phi_train = phi[2*i]
        phi_cv = phi[2*i + 1]
        target_cv = target.iloc[index[i]]
        target_train = target.loc[~target.index.isin(index[i])]
        w = np.linalg.inv(phi_train.values.T.dot(phi_train.values)).dot(phi_train.values.T).dot(target_train.values)
        nt = np.repeat(1,len(target_train))
        nc = np.repeat(1,len(target_cv))
        RMSE_train = (nt.dot((phi_train.values.dot(w) - target_train.values)** 2) *(1/len(target_train))) ** (1/2)
        RMSE_cv = (nc.dot((phi_cv.values.dot(w) - target_cv.values)** 2) *(1/len(target_cv))) ** (1/2)
        est = pd.DataFrame([w,RMSE_train, RMSE_cv]).transpose()
        ans = pd.concat([est,ans],axis = 0)
    return ans


### 分別計算M=2、M=1的train和cv之phi值以及RMSE，另外CV選擇5組###

#### M=2 ####

In [14]:
phi_2 = phi(train , 2, 5)
est_2 = estimate(phi_2, target)
est_2.columns = ['w', 'train_RMSE', 'cv_RMSE']
print(est_2.iloc[ : , 1:])

             train_RMSE               cv_RMSE
0  [3.3303181800916346]   [4.189775058922326]
0  [3.3093008606901155]   [4.600817032024187]
0  [3.1960605259660264]  [4.6990249675649896]
0   [3.340694611128524]   [4.775224042428947]
0  [3.1859276161398653]   [5.502359841588144]


在M=2的情況下，可以看到train的Rmse在[3.2,3.3]之間，CV的部分RMSE卻比train大很多，這表示我們的model存在著over-fitting的問題

#### M=1 ####

In [15]:
phi_1 = phi(train , 1, 5)
est_1 = estimate(phi_1, target)
est_1.columns = ['w', 'train_RMSE', 'cv_RMSE']
est_1.index = ['1', '2', '3', '4', '5']
est_1 = est_1.astype({"train_RMSE": float, "cv_RMSE": float})
print(est_1.iloc[ : , 1:])

   train_RMSE   cv_RMSE
1    4.010777  4.541331
2    4.089115  4.237307
3    4.100977  4.157073
4    4.141310  4.006769
5    4.147419  3.965697


在M=1的情況下，可以看到在train的RMSE相較於M=2的情況來說比較高，原因是我們的model相對簡單，但是與CV的RMSE與train的結果一致，所以在M=1的model中，沒有over-fitting的問題

## 2.1.(b) ##

###   Please analyze the weights of polynomial models for M = 1 and select the most contributive attribute which has the lowest RMS error on the Training Dataset. ###

In [17]:
print(est_1.iloc[ : , 1:])
min = int(est_1[['cv_RMSE']].idxmin())
print('cv_RMSE平均最小的S為第{num}組，RMSE = {n}'.format(num = int(est_1[['cv_RMSE']].idxmin()), n = est_1.iloc[min-1,2]))

   train_RMSE   cv_RMSE
1    4.010777  4.541331
2    4.089115  4.237307
3    4.100977  4.157073
4    4.141310  4.006769
5    4.147419  3.965697
cv_RMSE平均最小的S為第5組，RMSE = 3.9656974157022034


### 接著我們將這一組的係數取出來 ###

In [18]:
print(est_1.iloc[min-1,0])

[[-2.54950233e+01]
 [ 4.01963036e-02]
 [ 2.59312214e+01]
 [ 2.11421770e+01]
 [-2.56897128e+01]
 [ 1.35691812e+00]
 [ 1.89115549e+00]
 [-1.68807135e+00]
 [ 2.05822229e-02]
 [ 4.18774094e-01]
 [-9.77538251e-01]
 [ 6.95351852e-02]
 [ 3.87195305e-01]
 [-1.68751089e+01]
 [ 3.75798039e-02]
 [-3.16457581e-02]
 [ 1.79422061e+00]
 [-3.33245043e+00]]


## 2.2 ##

### (a) Choose some of air quality measurement in dataset X.csv and design your model. ### 

選擇Gaussian basis function，並挑選11個解釋變數進行分析 

In [19]:
var = list(train.columns)[:11]
print(var)

['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'RAINFALL', 'RH']


### 找出各變數的mean($\mu_j $)為何 ###

In [20]:
mean_v = list()
for i in range(len(var)) :  
    mu = np.mean(train[var[i]])
    mean_v.append(mu)

### 定義Gaussian basis function 和design matrix ### 

$$Gaussian : \phi_j(x) = exp{(\frac{(x-\mu_j)^2}{2S^2})}$$

Where the μj govern the locations of the basis functions in input space, and the parameter s governs their spatial scale.

In [21]:
def gaussian_bf (x, sigma) : 
    fun = lambda x : np.exp(-0.5 * (x - np.mean(x)) ** 2 / sigma ** 2)
    phi = x.apply(fun)
    return phi

### 定義函數：係數估計(w) 以及 RMSE ###

In [39]:
def estimate_g(phi, train, target) : 
    w = np.linalg.inv(phi.values.T.dot(phi.values)).dot(phi.values.T).dot(target.values)
    n = np.repeat(1,len(train))
    RMSE = (n.dot((phi.values.dot(w) - target.values)** 2) *(1/len(train))) ** (1/2)
    return [w, RMSE]

In [46]:
phi_g = gaussian_bf(train, 0.1).loc[ : ,var]

In [48]:
train_G = train[var]
estimate_g(phi_g,train_G , target)

[array([[ 11.13008978],
        [  4.32191387],
        [-10.89412996],
        [ 20.65122188],
        [ -1.19395632],
        [  1.62555175],
        [ -1.383968  ],
        [  1.50848435],
        [  0.61093422],
        [  6.69402259],
        [  1.97055026]]), array([10.75947448])]

可以看到我們係數的估計為何，以及RMSE

### (b) Apply N-fold cross-validation in your training stage to select at least one hyperparameter ###

### 我所選擇的hyperparameter為 gaussian fuction裡面的S(sigma)  ###

### 首先先將資料切成5份 ### 

In [49]:
index = split(5)

### s的範圍為 0.1 ~ 0.5 ，總共測試5個s，透過上面定義的函數來計算 $\phi(x)$ 以及各自的RMSE（產生5個dataframe，裡面分別包含5-fold的RMSE結果） ###

In [50]:
s = np.linspace(0.1, 0.5 ,5)

In [59]:
for k in range(len(s)) :  
    ansG_t = pd.DataFrame()
    ansG_cv = pd.DataFrame()
    for j in range(5) : 
        train_g = train_G.loc[~train.index.isin(index[j])]
        train_g_t = target.loc[~train.index.isin(index[j])]
        cv_g = train_G.loc[train.index.isin(index[j])]
        cv_g_t = target.loc[train.index.isin(index[j])]
        phi = pd.DataFrame()
        phicv = pd.DataFrame()
        for i in range(len(var)) : 
            phi_t = pd.DataFrame(gaussian_bf(pd.DataFrame(train_g.loc[ : ,var[i:i+1]]), s[k]))
            phi = pd.concat([phi, phi_t], axis = 1 , ignore_index= True)
            phi_cv = pd.DataFrame(gaussian_bf(pd.DataFrame(cv_g[var[i]]), s[k]))
            phicv = pd.concat([phicv, phi_cv], axis = 1 , ignore_index= True)
        ansg_t = estimate_g(phi ,train_g , train_g_t)
        ansg_cv = estimate_g(phicv , cv_g , cv_g_t)
        ansG_t =  pd.concat([pd.DataFrame(ansg_t[ : ]).T,ansG_t],axis = 0)                              
        ansG_cv = pd.concat([pd.DataFrame(ansg_cv[ : ]).T,ansG_cv],axis = 0)
        ansG = pd.concat([ansG_t,ansG_cv],axis = 1)
    ansG.columns = ['w', 'train_RMSE','w','cv_RMSE']
    ansG.index = ['1', '2', '3', '4', '5']
    globals()["ansG" + str(k + 1)] = ansG

### 針對對應的S值所記算出來的RMSE矩陣，計算cv_RMSE的平均，並找出cv_RMSE平均最小的S為何 ###

In [86]:
mean = pd.DataFrame()
for i in range(len(s)) :  
    x = pd.DataFrame([np.mean(globals()["ansG" + str(i + 1)].iloc[:, 3:])])
    mean = pd.concat([x, mean], axis = 0)
mean.index = ['1', '2', '3', '4', '5']
min = pd.DataFrame(mean[['cv_RMSE']].idxmin())
print(mean)
print('RMSE平均最小的S為{num}，RMSE = {n}'.format(num = s[int(min.iloc[0,0])-1] , n = np.min(mean.iloc[:,0])))

     cv_RMSE
1   8.999457
2   9.035532
3   9.089470
4   9.299276
5  10.592466
RMSE平均最小的S為0.1，RMSE = 8.999457309102066


## 3. Maximum a posteriori(MAP) approach ##

### Use maximum a posteriori approach method and repeat 2 . You could choose Gaussian distribution as a prior ###

## (a) 

Gaussian noise model:  
$$\epsilon \sim N(0,\ \beta)$$  
透過以下的公式更新我們的參數:   

$$p(w|t) = N(w|m_N,\ S_N)\ \rm{,where\ }$$  
$$S_N^{-1} = S_0^{-1} + \beta \Phi^T \Phi$$  
$$m_N = S_N (S_0^{-1}m_0 + \beta \Phi^T t)$$

建立我們的起始參數：$m_0$、$S_0$和$\beta$

In [87]:
m0 = np.repeat(0,len(var))
s0 = np.eye(len(var))
b = 1 
print(m0)
print(s0)

[0 0 0 0 0 0 0 0 0 0 0]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 定義函數：$p(w|t)$

In [65]:
def posterior(x, t, sigma , m0, s0, b) : 
    t = t.values
    phi = gaussian_bf (x, sigma)
    sn = np.linalg.inv(np.linalg.inv(s0) + b * phi.values.T.dot(phi.values))
    mn = sn.dot(np.linalg.inv(s0).dot(m0).reshape(11,1) + b * phi.values.T.dot(t))
    return[sn , mn]

In [66]:
r = posterior(train.loc[: , var],target,0.1,m0,s0,b)
r1 = r[0]

#### 將資料切分成50份

## MAP approach

In [67]:
def Map(x, t , sigma , n ,m0, s0, b) : 
    train = x.loc[ : ,var]
    for i in range(n-1) : 
        x = train.iloc[index[i], : ]
        t = target.iloc[index[i], : ]
        r = posterior(x, t, 0.1, m0, s0, b)
        m0 = r[1]
        s0 = r[0]
    x = train.iloc[index[49], : ]
    t = target.iloc[index[49], : ]
    r = posterior(x, t, sigma, m0, s0, b)
    return r 
        

In [68]:
index = split(100)

In [69]:
m0 = np.repeat(0,len(var))
s0 = np.eye(len(var))
b = 1 
r = Map(train.loc[ : ,var], target , 0.2, 100, m0 , s0 , b)

In [70]:
n = np.repeat(1,len(train))
phi = gaussian_bf (train.loc[ : ,var], 0.2)
print((n.dot((phi.values.dot(r[1]) - target.values)** 2) *(1/len(train))) ** (1/2))

[9.79628501]


## (b) ##

測試不同的S : (0.1 ,0.2 ,0.3 ,0.4, 0.5)

s = 0.1 

In [79]:
r1 = Map(train.loc[ : ,var], target , 0.1, 100, m0 , s0 , b)
phi_1 = gaussian_bf (train.loc[ : ,var], 0.1)
hyper1 = float((n.dot((phi_1.values.dot(r1[1]) - target.values)** 2) *(1/len(train))) ** (1/2))
hyper = [hyper1]
print('RMSE為{num}'.format(num = hyper1))


RMSE為10.851152076017991


s = 0.2 

In [80]:
r2 = Map(train.loc[ : ,var], target , 0.2, 100, m0 , s0 , b)
phi_2 = gaussian_bf (train.loc[ : ,var], 0.2)
hyper2 = float((n.dot((phi_2.values.dot(r2[1]) - target.values)** 2) *(1/len(train))) ** (1/2))
hyper.append(hyper2)
print('RMSE為{num}'.format(num = hyper2))


RMSE為9.796285010886661


s = 0.3 

In [81]:
r3 = Map(train.loc[ : ,var], target , 0.3, 100, m0 , s0 , b)
phi_3 = gaussian_bf (train.loc[ : ,var], 0.3)
hyper3 = float((n.dot((phi_3.values.dot(r3[1]) - target.values)** 2) *(1/len(train))) ** (1/2))
hyper.append(hyper3)
print('RMSE為{num}'.format(num = hyper3))

RMSE為9.748188501220547


s = 0.4 

In [82]:
r4 = Map(train.loc[ : ,var], target , 0.4, 100, m0 , s0 , b)
phi_4 = gaussian_bf (train.loc[ : ,var], 0.4)
hyper4 = float((n.dot((phi_4.values.dot(r4[1]) - target.values)** 2) *(1/len(train))) ** (1/2))
hyper.append(hyper4)
print('RMSE為{num}'.format(num = hyper4))

RMSE為9.784211942465895


s = 0.5 

In [83]:
r5 = Map(train.loc[ : ,var], target , 0.5, 100, m0 , s0 , b)
phi_5 = gaussian_bf (train.loc[ : ,var], 0.5)
hyper5 = float((n.dot((phi_5.values.dot(r5[1]) - target.values)** 2) *(1/len(train))) ** (1/2))
hyper.append(hyper5)
print('RMSE為{num}'.format(num = hyper5))

RMSE為9.810886684446048


## 找出具有最小RMSE的S

In [84]:
hyper = pd.DataFrame(hyper)
hyper.columns = ['RMSE']
print('第{num}組S有最小的RMSE ={num2}'.format(num = int(hyper[['RMSE']].idxmin())+1 ,num2 = hyper.iloc[int(hyper.idxmin()),0]))

第3組S有最小的RMSE =9.748188501220547
